In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD, accuracy
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate, train_test_split
from sklearn.model_selection import ParameterGrid
import pickle
from tqdm import tqdm



# 1. 데이터 로딩 10005  a 노래듣 듣는 다면 a300 번 노래를 추천해주느게 좋다 
file_path = '/home/ubuntu/project/4.5HZ/Data/완성/플레이리스트/플레이리스트_협업필터링(12월26일).csv'
df = pd.read_csv(file_path, encoding='utf-8')

# 2. 정보 확인
print(df.info())
print(df.isnull().sum())
print(df.shape)

# 3. 데이터 전처리
reader = Reader(rating_scale=(0, 5))  # rating 점수에 따라  평점 범위 0 ~ 5 로 지정
data = Dataset.load_from_df(df[['PLAYLIST_ID', 'TRACK_ID', 'rating']], reader)  # surprise 라이브러리가 사용할 수 있는 형태로 변환

# 4. 모델 학습
model = SVD()
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)  # cross_validate 함수를 이용해서 데이터를 5개로 구분하고 각각의 부분을 한번씩 테스트 세트로 사용해서 모델을 학습 및 평가

# 5. 모델 테스트
trainset, testset = train_test_split(data, test_size=.25)
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions)

# 6. 그리드서치 통한 하이퍼파라미터 최적화
#param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.001, 0.002, 0.005], 'reg_all': [0.2, 0.4, 0.6]}
param_grid = {'n_epochs': [5], 'lr_all': [0.001], 'reg_all': [0.2]} # 코드 확인용
grid = ParameterGrid(param_grid)

best_params = {}
best_score = float('inf')

for params in tqdm(grid):
    algo = SVD(**params)
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5)
    mean_rmse = cv_results['test_rmse'].mean()

    if mean_rmse < best_score:
        best_score = mean_rmse
        best_params = params

print(f"Best params: {best_params}, Best score: {best_score:.4f}")

# 7. 최적의 하이퍼파라미터를 사용하여 SVD 모델 학습
algo = SVD(**best_params)
trainset, testset = train_test_split(data, test_size=.25)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions) 

# 8. 전체 데이터로 최종 모델 학습
full_trainset = data.build_full_trainset()
algo.fit(full_trainset)

# 9. 모델 저장
pickle.dump(algo, open('Collaborative_Filtering_model_test.pkl', 'wb'))
